# [AI训练营]PaddleX实现目标检测baseline

手把手教你基于PaddleX实现目标检测。你需要实现以下任务：

> 1. 配置数据集（数据集选择、数据处理）
> 2. 配置模型并训练
> 3. 项目跑通即可达到结业要求

# 一、数据集说明

本项目使用的数据集是：[[AI训练营]目标检测数据集合集](https://aistudio.baidu.com/aistudio/datasetdetail/103743)，包含口罩识别 、交通标志识别、火焰检测、锥桶识别以及中秋元素识别。

该数据集已加载至本环境中，位于：**data/data103743/objDataset.zip**

In [ ]:
# 解压数据集（解压一次即可，请勿重复解压）
!unzip -oq /home/aistudio/data/data103743/objDataset.zip

解压完成后，左侧文件夹处会多一个名为**objDataset**的文件夹，该文件夹下有5个子文件夹：
- **barricade**——Gazebo锥桶检测
- **facemask**——口罩检测
- **fire**——火焰检测
- **MidAutumn**——中秋元素检测
- **roadsign_voc**——交通路标检测

每个子文件夹下有2个文件夹，分别存放着图像（**JPEGImages**）和标注文件（**Annotations**），如下所示：

In [ ]:
# 查看数据集文件结构
!tree objDataset -L 2

objDataset
├── barricade
│   ├── Annotations
│   └── JPEGImages
├── facemask
│   ├── Annotations
│   └── JPEGImages
├── fire
│   ├── Annotations
│   └── JPEGImages
├── MidAutumn
│   ├── Annotations
│   └── JPEGImages
└── roadsign_voc
    ├── Annotations
    ├── JPEGImages
    ├── labels.txt
    ├── test_list.txt
    ├── train_list.txt
    └── val_list.txt

15 directories, 4 files


# 二、数据准备

本基线系统使用的数据格式是PascalVOC格式，开发者基于PaddleX开发目标检测模型时，无需对数据格式进行转换，开箱即用。

但为了进行训练，还需要将数据划分为训练集、验证集和测试集。划分之前首先需要**安装PaddleX**。

In [1]:
# 安装PaddleX
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 31.1MB/s ta 0:00:01
     |████████████████████████████████| 1.0MB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 153kB 29.1MB/s eta 0:00:01
     |████████████████████████████████| 215kB 23.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.7MB/s ta 0:00:01
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


使用如下命令即可将数据划分为70%训练集，20%验证集和10%的测试集。

In [2]:
# 划分数据集
!paddlex --split_dataset --format VOC --dataset_dir objDataset/roadsign_voc --val_value 0.2 --test_value 0.1

Dataset Split Done.
Train samples: 615
Eval samples: 175
Test samples: 87
Split files saved in objDataset/roadsign_voc


划分完成后，该数据集下会生成**labels.txt**, **train_list.txt**, **val_list.txt**和**test_list.txt**，分别存储类别信息，训练样本列表，验证样本列表，测试样本列表。如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/d29a92b4cfc34b0097ef46dbbc8562af824387889f224948ae49283e0adee19d)

在这里，**你需要将path to dataset部分替换成你选择的数据集路径**。在左侧文件夹处，将鼠标放到你想选择的数据集文件夹上，会出现三个图标，第一个图标表示复制该文件夹路径，点击即可获得该文件夹路径，用这个路径替换path to dataset即可。

![](https://ai-studio-static-online.cdn.bcebos.com/c28ed88586644f64b34709a592fea0b97ec80470c0e041fd9aa6b8da21c8e283)


# 三、数据预处理

在训练模型之前，对目标检测任务的数据进行操作，从而提升模型效果。可用于数据处理的API有：
- **Normalize**：对图像进行归一化
- **ResizeByShort**：根据图像的短边调整图像大小
- **RandomHorizontalFlip**：以一定的概率对图像进行随机水平翻转
- **RandomDistort**：以一定的概率对图像进行随机像素内容变换

更多关于数据处理的API及使用说明可查看文档：
[https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html](https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html)

In [3]:
from paddlex.det import transforms

# 定义训练和验证时的transforms
# API说明 https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/det_transforms.html
train_transforms = transforms.Compose([
    # 此处需要补充图像预处理代码
    transforms.Normalize(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomDistort()
])

eval_transforms = transforms.Compose([
    # 此处需要补充图像预处理代码
    transforms.Normalize(),
    transforms.Resize(26)
])

读取PascalVOC格式的检测数据集，并对样本进行相应的处理。

In [4]:
import paddlex as pdx

# 定义训练和验证所用的数据集
# API说明：https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-vocdetection
train_dataset = pdx.datasets.VOCDetection(
    data_dir='objDataset/roadsign_voc',
    file_list='objDataset/roadsign_voc/train_list.txt',
    label_list='objDataset/roadsign_voc/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='objDataset/roadsign_voc',
    file_list='objDataset/roadsign_voc/val_list.txt',
    label_list='objDataset/roadsign_voc/labels.txt',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


2021-08-15 08:34:02 [INFO]	Starting to read file list from dataset...
2021-08-15 08:34:03 [INFO]	615 samples in file objDataset/roadsign_voc/train_list.txt
creating index...
index created!
2021-08-15 08:34:03 [INFO]	Starting to read file list from dataset...
2021-08-15 08:34:03 [INFO]	175 samples in file objDataset/roadsign_voc/val_list.txt
creating index...
index created!


需要注意的是：
- **data_dir** (str): 数据集所在的目录路径。
- **file_list** (str): 描述数据集图片文件和对应标注文件的文件路径（文本内每行路径为相对data_dir的相对路径）。
- **label_list** (str): 描述数据集包含的类别信息文件路径。

需要将第二步数据准备时生成的labels.txt, train_list.txt, val_list.txt和test_list.txt配置到以上变量中，如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/6462f7811da3436290948e5dde0c497d6ae51bcfe1904e0a863ff032363a4448)


# 四、模型训练

PaddleX目前提供了FasterRCNN和YOLOv3两种检测结构，多种backbone模型。本基线系统以骨干网络为DarkNet53的YOLOv3算法为例。

In [5]:
# 初始化模型
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#paddlex-det-yolov3

# 此处需要补充目标检测模型代码
model = pdx.det.YOLOv3(num_classes=len(train_dataset.labels), backbone='DarkNet53')

In [8]:
# 模型训练
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#id1
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html

# 此处需要补充模型训练参数
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=16,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_dir='output/yolov3_mobilenetv1')


2021-08-15 09:30:35 [ERROR]	Function model.train() only can be called once in your code.


SystemExit: -1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
model.predict( "objDataset/roadsign_voc/JPEGImages/road110.png")



[]

# 五、作业提交

完成以上四个步骤，并且跑通后，恭喜你！你已经学会使用PaddleX训练目标检测模型了！

首先点击右上方的“**文件**”，点击“**导出Notebook为Markdown**”：
![](https://ai-studio-static-online.cdn.bcebos.com/0b502ce690274eee89c285c702a26a72ff012df791eb45b79b7866a97f549e33)

导出后的文件用于上传至GitHub。

下面请点击左侧“版本”，在出现的新界面中点击“生成新版本”，按如下操作即可生成版本，用于提交作业：
![](https://ai-studio-static-online.cdn.bcebos.com/4e58b131f2db45289119dfa90081eea43b781d6894254069b4f02cffa067ffe1)

生成版本后，回到项目主页，点击“**设置为公开**”：
![](https://ai-studio-static-online.cdn.bcebos.com/4a9229e9eb8146169c848327ccb1f85ea30af431eee14aea9c2e6c9aeb364926)
